<a href="https://colab.research.google.com/github/RaghavDM1976/IK_Python/blob/main/Building_RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building a RAG Application: A Comprehensive Demo**

In today's digital healthcare landscape, patients and medical professionals frequently seek accurate, up-to-date answers to a wide range of medical questions. However:

- Search engines return too much noise or unverified sources.

- Chatbots often hallucinate or give generic advice without referencing valid information.

- Medical professionals/Patients have limited time to find contextually relevant evidence or literature.

There is a growing need for an AI system that can:

- Provide precise, evidence-backed answers,

- Maintain context awareness during multi-turn conversations,

- And optionally enhance its knowledge using trusted real-time web sources.

### **Objective:**
In this notebook, you will explore the Retrieval-Augmented Generation (RAG) framework and its application to Large Language Models (LLMs), specifically in the context of a Medical Assistant Chatbot.

You will:

- **Set up the necessary libraries** and tools for implementing RAG with LLMs.

By the end of this demo, you will understand the RAG framework and its practical applications in enhancing LLMs.

\

---

\

### **What is RAG?**

**Retrieval-Augmented Generation (RAG)** is an advanced method to enhance **LLMs** with external knowledge. Instead of relying solely on the model's pre-trained knowledge, RAG allows the LLM to retrieve relevant information from external databases (like vector stores) during inference, making it suitable for dynamic and domain-specific applications.

RAG addresses many challenges faced by LLMs:
- **Domain Knowledge Gaps**: LLMs can be limited in specific knowledge areas, especially in evolving fields.
- **Factuality Issues & Hallucinations**: RAG helps reduce incorrect or fabricated answers by retrieving context from trusted data sources.
- **Real-Time Updates**: RAG enables the integration of continuously updated external knowledge without retraining the model.

\

---

\


### **Why Use RAG?**

RAG enhances the capabilities of LLMs by:
- **Improving Response Quality**: Provides context from up-to-date knowledge, making the response more accurate.
- **Handling Knowledge Gaps**: Allows LLMs to access external databases, which are often required for specialized tasks.
- **Reducing Hallucinations**: RAG reduces the risk of LLMs making up answers or providing misleading responses.
- **Faster Deployment**: RAG avoids the need for retraining the model for every new dataset or domain.

> For more detailed information, you can refer to this [RAG Paper](https://arxiv.org/abs/2312.10997) or the [Retrieval-Augmented Generation Blog](https://www.promptingguide.ai/research/rag.en#introduction-to-rag).



### **Step 1. Setup and Install Dependencies**

> Run the following cell to install dependencies required:

In [8]:
%%time

# 🧹 Clean up anything that could interfere
!pip uninstall -y langchain langchain-core langchain-community langchain-openai langchain-huggingface openai gradio transformers faiss-cpu sentence-transformers > /dev/null

# Upgrade tooling
!pip install -q --upgrade pip setuptools wheel

# Install compatible, verified versions step by step
!pip install -q langchain-core==0.3.15
!pip install -q langchain==0.3.15
!pip install -q langchain-community==0.3.15
!pip install -q langchain-openai==0.2.11
!pip install -q langchain-huggingface==0.3.1
!pip install -q openai==1.58.1
!pip install -q transformers==4.46.2
!pip install -q faiss-cpu==1.8.0.post1
!pip install -q gradio==4.44.0 jedi==0.19.1

# ✅ Verify that everything now imports correctly
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

print("✅ Everything installed and imported successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires transformers, which is not installed.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires transformers, which is not installed.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.15 which is incompatible.
langchain-text-splitters 0.3.11 requires langchain-core<2.0.0,>=0.3.75, but you have langchain-core 0.3.15 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all

In [ ]:
# 💻🔄 Restart runtime to reload environment
import os
os._exit(0)

In [1]:
# Import the warnings module and suppress any warnings that might appear during execution
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings to keep the output clean and focused

- The `warnings` module is used to manage and control the warning messages in Python.
- `filterwarnings("ignore")` tells Python to ignore all warning messages that may be raised during execution. This is often done to keep the notebook or script output cleaner, especially when warnings are known and not critical to the code's functionality.

### **Step 2. Imports and Configuration**

In [1]:
import os
# import openai
from openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import getpass

### **Step 3. Add OpenAI API Key**

[Models available](https://platform.openai.com/docs/models)

#### **📌 Best Practices for API Key Safety**


1. Always use a unique API key for each team member on your account
2. Never deploy your key in client-side environments like browsers or mobile apps
3. Never commit your key to your repository
4. Use Environment Variables in place of your API key
5. Use a Key Management Service
6. Monitor your account usage and rotate your keys when needed



In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:10]} ...")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# os.environ['OPENAI_API_KEY'] = getpass.getpass()

OPENAI_API_KEY: sk-proj-Oo ...


In [3]:
client = OpenAI()

#### **🛡️ Further Details**

1. Use .env Files for Local Development
  - Store the API key in a .env file and load it using dotenv.

    ```
    from dotenv import load_dotenv
    load_dotenv()
    API_KEY = os.getenv("OPENAI_API_KEY")
    ```
2. Use Secret Managers for Production
  - AWS: AWS Secrets Manager
  - Azure: Azure Key Vault
  - GCP: Google Secret Manager

3. Use Token-Based Authentication if Available
  - Some cloud services provide temporary tokens instead of static API keys.

### **Step 4. Text Cleaning and Preprocessing**

> We now clean and preprocess the data. The text needs to be formatted before it can be processed into vector embeddings.

#### **1. Upload the Dataset**
> (dataset.txt)

In [4]:
# from google.colab import files

# # Upload the CSV files
# uploaded = files.upload()

# # Verify files uploaded successfully
# print(uploaded)

In [5]:
!gdown 1727UCcg_Nxn9Nbj_LiX8pvGEpo591oxc

Downloading...
From: https://drive.google.com/uc?id=1727UCcg_Nxn9Nbj_LiX8pvGEpo591oxc
To: /content/ai-medical-chatbot.txt
100% 16.0k/16.0k [00:00<00:00, 36.8MB/s]


In [6]:
# Read and clean the dataset
with open('ai-medical-chatbot.txt', 'r') as f:
    data = f.read()

data = data.replace('\n\n','\n') # Clean unnecessary line breaks
data = data.split('---') # Split the data into separate sections based on a delimiter

In [7]:
len(data)

5

In [8]:
print(data[3])


# Medical History Assessment
### **What is Medical History Assessment?**
Medical History Assessment is a comprehensive evaluation process used to collect detailed information about a patient's past and current health conditions, family medical history, medications, and lifestyle factors for accurate diagnosis and treatment planning.
### **Information Required for Medical History Assessment**
Here are the essential details that should be provided during Medical History Assessment:
1. Current symptoms and their duration
2. Past medical conditions and treatments
3. Current medications and supplements
4. Family medical history
5. Allergies and adverse reactions
6. Surgical history
7. Social history (smoking, alcohol, exercise)
8. Reproductive history (for women)
9. Immunization records



In [9]:
print(data[4])


# FAQs about Medical Conditions
**Is it necessary to see a doctor for minor symptoms?**
It depends on the symptoms and their persistence. However, it's better to consult a healthcare provider for proper evaluation, especially if symptoms persist or worsen, rather than self-diagnosing.
**Are chronic conditions different from acute conditions?**
Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
**Can I get medical consultation from another state?**
Yes, you can receive medical consultation from healthcare providers in another state through telemedicine services. However, some states have specific licensing requirements for out-of-state medical practice.
**Is medical consultation mandatory for prescription medications?**
Yes, prescription medications require a valid prescription from a license

In [10]:
# Preprocess the text (adjust formatting for clean text)
for i in range(0, len(data)):
    if i==4:
      data[i] = data[i].replace('\n**', '\n###').replace('**','')
    else:
      data[i] = data[i].replace('**','')

In [11]:
print(data[3])


# Medical History Assessment
### What is Medical History Assessment?
Medical History Assessment is a comprehensive evaluation process used to collect detailed information about a patient's past and current health conditions, family medical history, medications, and lifestyle factors for accurate diagnosis and treatment planning.
### Information Required for Medical History Assessment
Here are the essential details that should be provided during Medical History Assessment:
1. Current symptoms and their duration
2. Past medical conditions and treatments
3. Current medications and supplements
4. Family medical history
5. Allergies and adverse reactions
6. Surgical history
7. Social history (smoking, alcohol, exercise)
8. Reproductive history (for women)
9. Immunization records



In [12]:
print(data[4])


# FAQs about Medical Conditions
###Is it necessary to see a doctor for minor symptoms?
It depends on the symptoms and their persistence. However, it's better to consult a healthcare provider for proper evaluation, especially if symptoms persist or worsen, rather than self-diagnosing.
###Are chronic conditions different from acute conditions?
Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
###Can I get medical consultation from another state?
Yes, you can receive medical consultation from healthcare providers in another state through telemedicine services. However, some states have specific licensing requirements for out-of-state medical practice.
###Is medical consultation mandatory for prescription medications?
Yes, prescription medications require a valid prescription from a licensed he

In [13]:
# Organize the data into question-answer pairs
ques_ans = dict()
for i in range(0, len(data)):
    topics = data[i].split('\n###')
    for topic in topics[1:]:
      question_answer_pair = topic.split('\n')
      ques_ans[question_answer_pair[0]] = " ".join(question_answer_pair[1:])

In [14]:
len(ques_ans)

48

In [15]:
ques_ans.keys()

dict_keys([' What are Medical Conditions?', ' Who needs Medical Consultation?', ' Types of Medical Conditions', ' Why do patients need early diagnosis?', ' Importance of Medical Consultation for different health conditions', ' How can patients seek new Medical Consultation', ' Documents required for new Medical Consultation', ' Cost of new Medical Consultation', ' Time required to get Medical Consultation', ' Information that can be updated in Medical Records', ' General process to update Medical Records', ' Documents required to update Medical Records', ' Documents required to update emergency contacts', ' Process to change primary care physician', ' Obtaining Medical Records', ' Documents required for obtaining Medical Records', ' Charges for Medical Records', ' Medical Test Coordination', ' Process for Medical Test Coordination', ' HealthCare fees for test coordination', ' Documents required for Medical Test Coordination', ' Time required for Medical Test coordination?', ' What is M

In [16]:
print(ques_ans[" Why do patients need early diagnosis?"])

Early diagnosis is crucial for optimal health outcomes. However, early medical intervention becomes particularly important in the following situations: 1. Early diagnosis is required to prevent progression of diseases such as cancer, cardiovascular disease, and autoimmune conditions where timely treatment significantly improves prognosis. 2. If a patient has family history of genetic disorders, they should undergo screening tests. Early diagnosis is necessary to implement preventive measures and lifestyle modifications. 3. If a patient wants to maintain optimal health and prevent complications, they must seek regular medical check-ups and screenings.


In [17]:
all_content = str()
for key, value in ques_ans.items():
    # print(key)
    all_content += key + " " + value + "\n"
# print(all_content)
# Print cleaned content for verification

In [18]:
print(all_content)

 What are Medical Conditions? Medical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.
 Who needs Medical Consultation? All individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.
 Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditions ar

#### **2. Text Chunking**

> To prepare the text for semantic search, we break the content into smaller chunks. This will make it easier to process and index efficiently.


In [19]:
from langchain.text_splitter import CharacterTextSplitter
# Split the large content into smaller chunks for indexing
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=300, chunk_overlap=128, length_function=len)

chunks = text_splitter.split_text(all_content)

# Display the first chunk for verification
print(chunks)

['What are Medical Conditions? Medical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.', 'Who needs Medical Consultation? All individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.', 'Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditio

In [20]:
len(chunks)

47

#### **4. Store Embeddings in FAISS**

> We use FAISS to store and index the vectors. FAISS is optimized for similarity search, allowing us to retrieve the most relevant vectors during the search phase.

In [21]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings()

vectorStore = FAISS.from_texts(chunks, embeddings)
vectorStore.save_local("faiss_doc_idx")

/tmp/ipython-input-3122267179.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


#### **5. Perform Semantic Search**

> Now that we have indexed the embeddings, we can perform a semantic search. This search will find the most relevant documents based on a query, considering the meaning rather than exact matches.

In [22]:
docs = vectorStore.similarity_search("What are the main differences between acute and chronic medical conditions?")

# """
# - What does abutment of the nerve root mean in MRI reports?
# - What is the treatment for L5-S1 disc bulge?
# - What does mild concentric left ventricular hypertrophy mean?
# - How can I lose weight with hypothyroidism?
# """

In [24]:
len(docs)

4

In [25]:
for doc in docs:
    print(doc.page_content)

Are chronic conditions different from acute conditions? Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditions are long-term health problems that persist over time and require ongoing management. They include diabetes, hypertension, arthritis, and other conditions that need continuous care and monitoring. Both acute and chronic conditions require proper medical evaluation and treatment. The a

In [26]:
docs = vectorStore.similarity_search("I have pain in arm, should I consult multiple doctors?")
for doc in docs:
    print(doc.page_content)

Is it necessary to see a doctor for minor symptoms? It depends on the symptoms and their persistence. However, it's better to consult a healthcare provider for proper evaluation, especially if symptoms persist or worsen, rather than self-diagnosing.
Can people with multiple conditions see different specialists? Yes, patients with multiple medical conditions often need care from various specialists. Care coordination between providers is important to avoid medication interactions and ensure comprehensive treatment.
Who needs Medical Consultation? All individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.
Can I use medical consultation for second opinions? Yes, seeking second opinions is a patient right and often recommended for serious diagnoses or major treatment decisions. Most insurance plans cover second opinion consultations.


#### 6. **Setup OpenAI Model**


In [27]:
from langchain_community.llms import HuggingFaceHub

In [28]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI

In [29]:
question = "What are the main differences between acute and chronic medical conditions?"


llm = OpenAI()
template = """You are a medical assistant chatbot helping answer patient questions based only on the provided context.
Do not guess or provide inaccurate information. If the answer is not found in the context, say you don’t know.
You will answer the question based on the context - {context}.
Question: {question}
Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Create Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorStore.as_retriever(search_type="similarity", k=4),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

# Run the chain with the user query
result = qa_chain({"query": question})
answer = result['result']

/tmp/ipython-input-748523771.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
/tmp/ipython-input-748523771.py:20: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [30]:
print("answer is\n", answer)

answer is
  The main difference between acute and chronic medical conditions is that acute conditions develop suddenly and require immediate medical attention, while chronic conditions are long-term health problems that require ongoing management. The approach to treatment and management also differs significantly between the two types of conditions.


In [31]:
print("answer is\n", answer)

answer is
  The main difference between acute and chronic medical conditions is that acute conditions develop suddenly and require immediate medical attention, while chronic conditions are long-term health problems that require ongoing management. The approach to treatment and management also differs significantly between the two types of conditions.


In [32]:
result.keys()

dict_keys(['query', 'result'])

### **Step 5. Gradio Chatbot**

In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
import traceback

# Toggle this flag to True to enable debug logs
DEBUG = True

def predict(message, history):
    try:
        if DEBUG: print(f"\n📥 Received message: {message}")
        history_langchain_format = []

        # Convert chat history into LangChain format
        for human, ai in history:
            history_langchain_format.append(HumanMessage(content=human))
            history_langchain_format.append(AIMessage(content=ai))

        # Define Prompt
        template = """You are a medical assistant chatbot helping answer patient questions based only on the provided context.
        Do not guess or provide inaccurate information. If the answer is not found in the context, say you don’t know.
        You will answer the question based on the context - {context}.
        Question: {question}
        Answer:"""
        QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

        # Create Retrieval QA Chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=vectorStore.as_retriever(search_type="similarity", k=4),
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )

        # Run the chain with the user query
        result = qa_chain({"query": message})
        answer = result['result']

        if DEBUG: print(f"✅ Retrieved Answer:\n{answer}")

        # Update history
        history_langchain_format.append(HumanMessage(content=message))
        history_langchain_format.append(AIMessage(content=answer))

        return answer

    except Exception as e:
        error_trace = traceback.format_exc()
        if DEBUG: print(f"❌ Exception occurred:\n{error_trace}")
        return "⚠️ Sorry, an internal error occurred. Please try again later."

# Gradio Chatbot UI
gr.ChatInterface(
    fn=predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question related to Healthcare and Medical Services", container=False, scale=7),
    title="DocumentQABot",
    theme="soft",
    examples=[
        "What are the main differences between acute and chronic medical conditions?",
        "What are the main differences between acute and chronic medical conditions?",
        "What does mild concentric LV hypertrophy mean?"
    ],
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://da670cf560195465a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---
#  -----------------------------------------------------  **THANK YOU** ------------------------------------------------------------


---